In [3]:
import csv
# emotions = list(emotions_keywords_df)[1:]
# emotion_keywords_csv = 'Andbrain_DataSet.csv'
data_path = '../../datasets/'
emotion_keywords_csv = 'andbrainDataSet108.csv'
emotions_keywords = {}
emotions2 = []
with open(data_path + emotion_keywords_csv, mode='r', errors='replace') as csvfile:
    reader = csv.reader(csvfile, dialect='excel')
    emotions2 = reader.__next__()  # skip headers
#     print (emotions2)
    for row in reader:
#         print(row)
        dico = {}
        for i in range(1,8):
            dico[emotions2[i].strip()] = float(row[i])
        emotions_keywords[row[0].strip()] = dico

global emotions
emotions = emotions2[1:]
emotions

['disgust', 'surprise', 'neutral', 'anger', 'sad', 'happy', 'fear']

In [17]:
support_keywords = {'dvd': 1, 'vhs': 1,'edition': 1, 'blue-ray': 1, 'blueray': 1, 'blu-ray': 1, 'bluray': 1, 'price': 1}

In [43]:
def fill_emotions(comment):
    comment_text = comment['reviewText']
    comment_words = comment_text.split(' ')
    emotions_found = False
    
    emotion_keywords_hits = 0
    scores = {key: 0.0 for key in emotions}    # {'disgust': 0.0, ...}
    
    for word in comment_words:
        low_word = word.lower()
        
        # detect and skip potentially support-related commments
        if support_keywords.get(low_word, 0) > 0:
            return None
        
        if emotions_keywords.get(low_word, 0.0) != 0.0:
            for key in scores.keys():
                scores[key] += emotions_keywords[low_word][key]
                emotions_found = True

    ## ignore the record if no emotions are found
    if emotions_found == False:
        return None
    
    for emotion in emotions:
        comment[emotion] = scores[emotion]
   
    return comment

In [47]:
test_comment = {'somekey': 'abc'}
test_comment['reviewText'] = 'This is scary movie I  ending'
res = fill_emotions(test_comment)
if res == None:
    print('Found support')
else:
    print(res)

{'somekey': 'abc', 'reviewText': 'This is scary movie I  ending', 'disgust': 0.036880419748299996, 'surprise': 0.1221477536738, 'neutral': 0.0425040389, 'anger': 0.039184567248299995, 'sad': 0.041502618503, 'happy': 0.0530233893315, 'fear': 0.14058789831500001}


In [50]:
import pandas as pd
import gzip

def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield eval(l)

def getDF(path, trunc=0):
  i = 0
  df = {}
  for d in parse(path):    
    res = fill_emotions(d)
    if res != None:
        df[i] = res
        i += 1
    if trunc > 0 and i > trunc: 
        break
  return pd.DataFrame.from_dict(df, orient='index')

data_path = '../../datasets/'
file_name = 'reviews_Movies_and_TV.json.gz'

In [59]:
# 1 million records in about 6 minutes
df = getDF(data_path + file_name, 1000)

In [60]:
len(df.index)

1001

In [58]:
## Saved one million records
# import pickle
# pickle_out = open(data_path + "amzn_1million_emotions.pickle","wb")
# pickle.dump(df, pickle_out)
# pickle_out.close()

In [61]:
# Reads one million in 5 seconds
pickle_in = open(data_path + "amzn_1million_emotions.pickle","rb")
df = pickle.load(pickle_in)

In [62]:
len(df.index)

1000001

In [63]:
# filter out if fewer than 10 reviews
movie_groups = df.groupby('asin')
movies_lots_comments = movie_groups.filter(lambda x: x['asin'].count() >= 10)
len(movies_lots_comments.index)

936935

In [64]:
movies_lots_comments.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,disgust,surprise,neutral,anger,sad,happy,fear
2,AP57WZ2X4G0AA,0000589012,Amy Arnett-German,"[0, 1]","The information is good, but the presentation ...",2.0,Boring,1366675200,"04 23, 2013",0.009882,0.101026,0.035483,0.004104,0.010058,0.023105,0.010120
3,A5Y15SAOMX6XA,0000589012,Catherine E. Wetzler,"[12, 16]","I, too, purchased this to use with my patients...",2.0,C+ At Best,1307404800,"06 7, 2011",0.303542,0.976544,0.387673,0.935520,0.734685,1.016630,0.901591
4,ANF0AGIV0JCH2,0000589012,GrammaCarole,"[13, 13]",My 18 year old grandson has been addicted to d...,5.0,Most Helpful Input I've received for my family,1308182400,"06 16, 2011",0.216804,0.590973,0.241898,0.592566,0.596843,0.894059,0.625119
5,A3LDEBLV6MVUBE,0000589012,"Jami Hyde ""Jami""","[10, 11]",The best book ever to help addicts and their f...,5.0,Why don't They Just Quit,1208995200,"04 24, 2008",0.168070,0.335279,0.093219,0.259617,0.326836,0.497257,0.348703
6,A2HWI21H23GDS4,0000589012,Jude R. Montarsi,"[1, 1]",The film contains a set of roundtable discussi...,4.0,Very good set of discussions,1338681600,"06 3, 2012",0.137722,0.114337,0.062523,0.122181,0.105733,0.331313,0.164324


In [71]:
vectors = movies_lots_comments.drop(['reviewerID','reviewerName','helpful','reviewText', 'summary', 'unixReviewTime', 'reviewTime'],axis=1)

In [72]:
vectors.head()

,asin,overall,disgust,surprise,neutral,anger,sad,happy,fear
2,0000589012,2.0,0.009882,0.101026,0.035483,0.004104,0.010058,0.023105,0.010120
3,0000589012,2.0,0.303542,0.976544,0.387673,0.935520,0.734685,1.016630,0.901591
4,0000589012,5.0,0.216804,0.590973,0.241898,0.592566,0.596843,0.894059,0.625119
5,0000589012,5.0,0.168070,0.335279,0.093219,0.259617,0.326836,0.497257,0.348703
6,0000589012,4.0,0.137722,0.114337,0.062523,0.122181,0.105733,0.331313,0.164324


In [73]:
len(vectors.index)

936935

In [77]:
def compute(group):
    return group.sum(axis=0) / group.count().iloc[1]
    
groups = vectors.groupby('asin')
avg_movies_emotions = groups.apply(compute)

In [78]:
avg_movies_emotions.head()

,anger,disgust,fear,happy,neutral,overall,sad,surprise
asin,,,,,,,,
0000589012,0.719303,0.314861,0.736618,0.873825,0.278869,4.083333,0.703079,0.737976
0005019281,0.270152,0.122634,0.269623,0.334098,0.138510,4.490909,0.246217,0.269665
0005119367,0.417027,0.203362,0.457906,0.599245,0.217509,4.804651,0.419634,0.493294
0073378208,0.200098,0.096303,0.227331,0.264090,0.131718,3.692308,0.200640,0.234587
0307141225,0.464897,0.207345,0.467378,0.659548,0.200352,4.727273,0.404058,0.565426
